# Lab 4: Advanced Neural Nets
### Шевченко Юлія, ФІ-31мн

### Lab Task:

1. _**Завдання щодо генерації текстів або машинного перекладу (на вибір) на базі рекурентних мереж або трансформерів (на вибір)**_. Вирішити завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.  
2. Провести експерименти з моделями бібліотеки HF Transformers за допомогою (наприклад) Pipeline модуля.
3. _**Завдання щодо генерації або стилізації зображень (на вибір)**_. Вирішити завдання перенесення стилю або генерації зображень (архітектура за вашим вибором: GAN/DCGAN/VAE).

In [1]:
# All the imports for the task
import os
import re
import string

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.src.preprocessing.image import ImageDataGenerator
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

### Task 1